In [1]:
from MODEL import MobileNetInspiredDetector
import torch
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import nms
import random
import json
import os
import numpy as np
from PIL import Image
from torch import topk
from torch.nn.functional import interpolate

In [3]:
weights = torch.load("ultramobileweights.ckpt",weights_only=True)

In [4]:
model = MobileNetInspiredDetector(num_classes=1,boxes = 5,input_ch=3)
model.load_state_dict(weights['model_state_dict'])

<All keys matched successfully>

In [5]:
ann_path = "D:/image_detection_data/annotation/annotation.json"
img_path = "D:/image_detection_data/images/"

In [6]:
ann = json.load(open(ann_path))

In [18]:
annotation = ann['annotations']
chosen_ann = random.choice(annotation)
id = chosen_ann['image_id']
img_name = ann['images'][id]['file_name'] 
# img_dir =  "C:/Users/Ankit Kumar/Downloads/man_img.png"
img_dir = img_path + img_name
box = chosen_ann['bbox']
box_t = torch.tensor(box)
print(box_t.shape)
img = Image.open(img_dir).convert('RGB')
img_arr = np.array(img)
img_tensor = torch.tensor(img_arr)
p_it = img_tensor.permute(2,0,1)
d_ip = draw_bounding_boxes(p_it,box_t,width=3)
d_arr = np.array(d_ip.permute(1,2,0), dtype=np.uint8)
d_img = Image.fromarray(d_arr)
d_img.show()


torch.Size([1, 4])


In [19]:
img_ = torch.tensor(img_arr, dtype=torch.float32)
out = model.forward(img_)
pred_box = out[0].squeeze(0)
pred_tgt = out[1].squeeze(0).squeeze(1)
s_box = torch.zeros_like(box_t)

k = topk(pred_tgt,1)
label = k[0].tolist()
lbl = [str(l) for l in label]
s_box[:,0] = box_t[:,0]*pred_box[k[1],0]
s_box[:,1] = box_t[:,1]*pred_box[k[1],1]
s_box[:,2] = box_t[:,2]*pred_box[k[1],0]
s_box[:,3] = box_t[:,3]*pred_box[k[1],1]
pd_tensor = draw_bounding_boxes(p_it,s_box , labels=lbl, width=4)
dp_arr = np.array(pd_tensor.permute(1, 2, 0), dtype=np.uint8)
dp_img = Image.fromarray(dp_arr)
dp_img.show()

In [9]:
s_box,box_t

(tensor([[109.1005,  89.7413, 326.0180, 513.9728]], grad_fn=<CopySlices>),
 tensor([[108.8000,  89.6815, 325.1200, 513.6306]]))

torch.Size([750, 2])

torch.return_types.topk(
values=tensor([0.5453], grad_fn=<TopkBackward0>),
indices=tensor([640]))